In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import time
from nltk.tokenize import TweetTokenizer
from gensim.parsing.porter import PorterStemmer

from sklearn.tree import DecisionTreeClassifier


In [3]:
tweets_df = pd.read_csv('/Users/arjunkhanchandani/Desktop/twitter_data_analysis-main/data/final_manual_priority.csv')
tweets_df.shape

(198, 19)

In [4]:
tweets_df.head()

,S.no.,user_id,tweet_id,username,location,following,followers,twt_created_at,total_tweets,retweet_count,text,hashtags,mentions,textblob_polarity,nltk_compound,avg_sentiment,textblob_sentiment,nltk_sentiment,priority
0,4,11,11,UtkarshMishra_9,"Noida, India",707,1122,2022-11-08 21:14:55+00:00,5764,0,estimated magnitude earthquake affected countr...,"[{'text': 'earthquake', 'indices': [137, 148]}...","[{'screen_name': 'ANI', 'name': 'ANI', 'id': 3...",0.000000,-0.1531,-1,0,-1,0
1,8,18,18,GirjeshKPatel,"‚Ä°¬ß‚â§‚Ä°¬ß√±‚Ä°¬ß¬Æ‚Ä°¬ß√§, ‚Ä°¬ß‚â†‚Ä°¬ß√¶...",164,988,2022-11-08 20:54:48+00:00,522,0,heavy roorke uttrakhand second horrible moment,"[{'text': 'earthquake', 'indices': [6, 17]}]","[{'screen_name': 'ZeeNews', 'name': 'Zee News'...",-0.400000,-0.5423,-1,-1,-1,1
2,10,25,25,TheAnantpandit,"New Delhi, India",409,19,2022-11-08 20:46:16+00:00,152,0,earthquake magnitude occurred ist lat long dep...,[],"[{'screen_name': 'Indiametdept', 'name': 'Indi...",-0.050000,0.0000,-1,-1,0,0
3,11,31,31,kanhagupta21,Allahabad,182,13,2022-11-08 20:38:44+00:00,73,0,horrible ended running outside home safe,[],"[{'screen_name': 'ANI', 'name': 'ANI', 'id': 3...",-0.166667,-0.1531,-1,-1,-1,1
4,13,42,42,Kunalgupta_voi,NaN,21,1,2022-11-08 20:10:38+00:00,1,0,choking wakeup antismog gun installed watering...,"[{'text': 'DelhiPollution', 'indices': [253, 2...","[{'screen_name': 'ArvindKejriwal', 'name': 'Ar...",0.000000,-0.6597,-1,0,-1,1


In [5]:
# dropping user_id, username, location, following, followers, twt_created_at, total_tweets, retweet_count, hashtags, mentions, tweet_id_dup
tweets_df.drop(['S.no.', 'user_id', 'username', 'location', 'following', 'followers', 'twt_created_at', 'total_tweets', 'retweet_count', 'hashtags', 'mentions', 'textblob_polarity', 'nltk_compound', 'textblob_sentiment', 'nltk_sentiment', 'avg_sentiment'], axis=1, inplace=True)

In [6]:
tweets_df.columns

Index(['tweet_id', 'text', 'priority'], dtype='object')

In [7]:
tweets_df.head()

,tweet_id,text,priority
0,11,estimated magnitude earthquake affected countr...,0
1,18,heavy roorke uttrakhand second horrible moment,1
2,25,earthquake magnitude occurred ist lat long dep...,0
3,31,horrible ended running outside home safe,1
4,42,choking wakeup antismog gun installed watering...,1


In [8]:
tweets_df['priority'].value_counts()

1    109
0     89
Name: priority, dtype: int64

In [9]:
def creating_tokens(df):
    tokens = list()
    tokenizer = TweetTokenizer()
    
    for tweets in df.loc[:, 'text']:
        # print(len(tokenizer.tokenize(tweets)))
        tokens.append(tokenizer.tokenize(tweets))
    
    df['tokens'] = tokens
    
    porter_stemmer = PorterStemmer()
    # Get the stemmed_tokens
    df['stemmed_tokens'] = [[porter_stemmer.stem(word) for word in tokens] for tokens in df['tokens']]
    df['stemmed_tokens'].head(10)
    
    
    return df

In [10]:
tweets_df = creating_tokens(tweets_df)
tweets_df.head()

,tweet_id,text,priority,tokens,stemmed_tokens
0,11,estimated magnitude earthquake affected countr...,0,"[estimated, magnitude, earthquake, affected, c...","[estim, magnitud, earthquak, affect, countri, ..."
1,18,heavy roorke uttrakhand second horrible moment,1,"[heavy, roorke, uttrakhand, second, horrible, ...","[heavi, roork, uttrakhand, second, horribl, mo..."
2,25,earthquake magnitude occurred ist lat long dep...,0,"[earthquake, magnitude, occurred, ist, lat, lo...","[earthquak, magnitud, occur, ist, lat, long, d..."
3,31,horrible ended running outside home safe,1,"[horrible, ended, running, outside, home, safe]","[horribl, end, run, outsid, home, safe]"
4,42,choking wakeup antismog gun installed watering...,1,"[choking, wakeup, antismog, gun, installed, wa...","[choke, wakeup, antismog, gun, instal, water, ..."


In [11]:
from sklearn.model_selection import train_test_split

def split_data(df, test_size):
    x_train, x_test, y_train, y_test = train_test_split(df['stemmed_tokens'], df['priority'], test_size=test_size, random_state=42, stratify=df['priority'])
    
    print(y_train.value_counts())
    print(y_test.value_counts())
    # print(type(x_train))
    # print(type(y_train))
    
    x_train = x_train.to_frame()
    x_train = x_train.reset_index()
    
    x_test = x_test.to_frame()
    x_test = x_test.reset_index()
    
    y_train = y_train.to_frame()
    y_train = y_train.reset_index()
    
    y_test = y_test.to_frame()
    y_test = y_test.reset_index()
    
    return x_train, x_test, y_train, y_test

In [12]:
x_train, x_test, y_train, y_test = split_data(tweets_df, 0.3)

1    76
0    62
Name: priority, dtype: int64
1    33
0    27
Name: priority, dtype: int64


In [13]:
print(x_train.head())
print(x_test.head())
print(y_train.head())
print(y_test.head())

   index                                     stemmed_tokens
0    164  [forgiv, rapist, murder, cruel, enter, mandir,...
1     36  [socha, puch, illeg, resort, demolish, still, ...
2     92  [todai, whole, year, complet, dai, written, pa...
3     42  [see, ground, realiti, statu, sardar, pathet, ...
4     50  [dirti, monei, aap, need, investig, lie, detec...
   index                                     stemmed_tokens
0    188  [hai, chor, sath, deta, hai, pich, leta, hai, ...
1     23  [travel, pass, markundi, toll, acp, toll, pvt,...
2    174  [final, todai, yet, mcd, remov, broken, bench,...
3     73  [crore, public, monei, invest, project, clean,...
4    152  [histor, judgement, suprem, court, put, stamp,...
   index  priority
0    164         0
1     36         0
2     92         0
3     42         1
4     50         0
   index  priority
0    188         1
1     23         1
2    174         0
3     73         1
4    152         0


In [105]:
from gensim.models import Word2Vec
import time

OUTPUT_FOLDER = '/Users/arjunkhanchandani/Desktop/twitter_data_analysis-main'

start_time = time.time()
tokens = pd.Series(tweets_df['stemmed_tokens']).values
# print(tokens)
word2vec_model_file = OUTPUT_FOLDER + 'word2vec_' + str(8000) + '.model'

w2v_model = Word2Vec(tokens, min_count=1, vector_size=8000, window=8, workers=7, sg=3)
print("Time taken to train word2vec model: " + str(time.time() - start_time))
w2v_model.save(word2vec_model_file)


Time taken to train word2vec model: 2.043767213821411


In [106]:
def create_file(create_file, model_file, x):
    sg_w2v_model = Word2Vec.load(model_file)
    
    with open(create_file, 'w+') as word2vec_file:
        for index, row in x.iterrows():
            model_vector = (np.mean([sg_w2v_model.wv[token] for token in row['stemmed_tokens']], axis=0)).tolist()
            if index == 0:
                header = ",".join(str(ele) for ele in range(8000))
                word2vec_file.write(header)
                word2vec_file.write("\n")
            
            if type(model_vector) is list:
                line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
            else:
                line1 = ",".join([str(0) for i in range(8000)])
            word2vec_file.write(line1)
            word2vec_file.write('\n')
    
    df = pd.read_csv(create_file)
    return df
        

In [107]:
word2vec_train_filename = OUTPUT_FOLDER + 'word2vec_train_' + str(8000) + '.csv'
word2vec_train_df = create_file(word2vec_train_filename, word2vec_model_file, x_train)
print(word2vec_train_df.shape)
word2vec_train_df.head()

(138, 8000)


,0,1,2,3,4,5,6,7,8,9,...,7990,7991,7992,7993,7994,7995,7996,7997,7998,7999
0,0.000089,-0.000029,0.000008,-0.000039,-0.000002,0.000010,-0.000026,0.000036,0.000017,0.000002,...,0.000022,-0.000019,0.000005,-0.000043,-0.000031,0.000013,-0.000023,0.000027,0.000004,0.000039
1,0.000128,-0.000029,0.000025,-0.000022,0.000047,-0.000011,-0.000050,0.000009,0.000059,-0.000004,...,0.000048,-0.000059,0.000061,0.000005,-0.000022,0.000035,-0.000026,-0.000006,-0.000008,0.000018
2,0.000156,-0.000052,0.000052,-0.000100,0.000077,0.000019,-0.000046,0.000057,0.000119,-0.000003,...,0.000065,-0.000115,0.000022,-0.000026,-0.000069,-0.000035,-0.000034,-0.000002,-0.000045,0.000075
3,0.000126,-0.000062,0.000037,-0.000113,0.000067,0.000014,-0.000051,0.000020,0.000090,0.000007,...,0.000031,-0.000109,0.000037,0.000007,-0.000044,-0.000024,-0.000029,0.000023,-0.000022,0.000047
4,0.000162,-0.000078,0.000076,-0.000148,0.000059,0.000013,-0.000082,0.000065,0.000126,0.000032,...,0.000070,-0.000098,0.000073,-0.000042,-0.000030,-0.000046,-0.000076,-0.000028,-0.000011,0.000069


In [108]:
word2vec_test_filename = OUTPUT_FOLDER + 'word2vec_test_' + str(8000) + '.csv'
word2vec_test_df = create_file(word2vec_test_filename, word2vec_model_file, x_test)
print(word2vec_test_df.shape)
word2vec_test_df.head()

(60, 8000)


,0,1,2,3,4,5,6,7,8,9,...,7990,7991,7992,7993,7994,7995,7996,7997,7998,7999
0,0.000154,-0.000046,0.000069,-0.000102,0.000052,0.000031,-0.000050,0.000059,0.000134,0.000038,...,0.000087,-0.000155,0.000067,-0.000044,-0.000040,0.000006,-0.000057,-0.000041,0.000011,0.000108
1,0.000093,-0.000040,0.000026,-0.000109,0.000053,0.000010,-0.000032,0.000047,0.000063,0.000015,...,0.000064,-0.000091,0.000028,-0.000032,-0.000030,-0.000006,-0.000021,-0.000025,0.000017,0.000093
2,0.000133,-0.000027,0.000045,-0.000042,0.000026,0.000037,-0.000020,0.000031,0.000057,0.000035,...,0.000056,-0.000066,0.000064,-0.000028,-0.000012,-0.000015,-0.000035,-0.000042,0.000008,0.000079
3,0.000147,-0.000069,0.000058,-0.000068,0.000103,0.000017,-0.000030,0.000063,0.000095,0.000006,...,0.000066,-0.000133,0.000075,-0.000003,-0.000052,-0.000023,-0.000023,-0.000033,-0.000029,0.000035
4,0.000084,-0.000044,0.000008,-0.000054,0.000006,0.000022,-0.000055,0.000014,0.000120,0.000002,...,0.000043,-0.000103,0.000042,-0.000038,-0.000054,0.000003,-0.000038,0.000007,-0.000038,0.000078


In [109]:
print(y_train.shape)
print(y_test.shape)

(138, 2)
(60, 2)


# Decision Tree

In [110]:
# Initialize the model
clf_decision_word2vec = DecisionTreeClassifier()

start_time = time.time()
# Fit the model
clf_decision_word2vec.fit(word2vec_train_df, y_train['priority'])
print("Time taken to fit the model with word2vec vectors: " + str(time.time() - start_time))

Time taken to fit the model with word2vec vectors: 0.2762019634246826


In [111]:
from sklearn.metrics import classification_report
        
y_pred_word2vec = clf_decision_word2vec.predict(word2vec_test_df)
print(classification_report(y_test['priority'], y_pred_word2vec))

              precision    recall  f1-score   support

           0       0.59      0.63      0.61        27
           1       0.68      0.64      0.66        33

    accuracy                           0.63        60
   macro avg       0.63      0.63      0.63        60
weighted avg       0.64      0.63      0.63        60



https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwitu6WthJ77AhX_TmwGHYzTBjMQFnoECBcQAQ&url=https%3A%2F%2Fmedium.com%2F%40zafaralibagh6%2Fa-simple-word2vec-tutorial-61e64e38a6a1&usg=AOvVaw3tHKEk24OxG_LwAiMr2wZs